## geoFiles_preparation
### This notebook generates the geoJSON, ZIP archives and geoparquet files related to counties/districts used in the OBI tool in the "Select an area" option
### last_processed_idx can be used to resume a failed computation

### Initial configuration
#### To start working with this particular notebook, you need to provide necessary credential and settings
#### Below is an template of configuration, which is necessary prepare aside of this notebook and copy & paste all content in triple quotes to the next cell's input field
    """
    {
    "COS_ENDPOINT_URL": "s3.private.eu-de.cloud-object-storage.appdomain.cloud",
    "COS_AUTH_ENDPOINT_URL": "https://iam.cloud.ibm.com/oidc/token",
    "COS_APIKEY": "xxx",
    "PRECREATED_GEOJSON_BUCKET": "counties-geojsons",
    "DB2_CONNECTION_STRING": "jdbc:db2://65beb513-5d3d-4101-9001-f42e9dc954b3.brt9d04f0cmqeb8u7740.databases.appdomain.cloud:30371/BLUDB:sslConnection=true;useJDBC4ColumnNameAndLabelSemantics=false;db2.jcc.charsetDecoderEncoder=3;",
    "DB2_USERNAME": "xxx",
    "DB2_PASSWORD": "xxx",
    "COUNTRY_TABLE": "FEATURES_DB_VIDA_EXTENDED",
    "SHARE_PARQUET_BUCKET": "districts-geoparquets"
    }
    """
    

In [2]:
# Read notebook configuration
import getpass
import json

config_str = getpass.getpass('Enter your prepared config: ')
config = json.loads(config_str)

In [3]:
# create a README.txt file so it can be added to the zip file later
readme_txt_text = '''
Attribute Dictionary:
- latitude --> latitude of the centroid of the building in degrees
- longitude --> longitude of the centroid of the building in degrees
- source --> Source of the building footprint (Google Open Buildings | Microsoft Building Footprints | OSM)
- urban_split --> Urban classification (Urban | Sub-urban | Rural)
- ghsl_smod --> Degree of Urbanisation derived by GHS-SMOD R2023A - GHS settlement layers
- res_type --> Residential Vs non-Residential building as per classification source
- type_source --> Source of the classification output (classification model | OSM derived)
- osm_type --> Tag if building classification is OpenStreetMap derived
- confidence --> Confidence value of the classification model (in percentage)
- height --> Height of the building in meters derived from WSF3DV3
- floors --> Estimated number of floors based on height of the building assuming ~3m/floor (first floor assumed ~4.5m)
- building_faces --> Estimated number of outer walls (faces) of the building (based on its geometry)
- perimeter_in_meters --> Estimated total length of the outer walls of the building in meters (based on its geometry)
- area_in_meters --> Estimated roof area of the building in square meters (when viewed from top based on its geometry)
- gfa_in_meters --> Gross floor area in square meters: Estimated based on rooftop area times the amount of floors
- elec_access_percent --> Estimated mean likelihood that the building of interest is connected to the electric grid, derived from Open Energy Maps
- elec_consumption_kWh_month --> Mean-point estimate of a modelled distribution curve of monthly electricity consumption for a building, given in kWh, derived from Open Energy Maps
- elec_consumption_std_kWh_month --> Standard deviation of the monthly electricity consumption value, derived from Open Energy Maps


Sources & References:
- Sentinel-2 Cloud-Optimized GeoTIFFs: Sentinel-2 satellite images are downloaded from the public S3 bucket Sentinel-2 Cloud-Optimized GeoTIFFs containing satellite images of the Earth’s surface divided into pre-defined tiles (https://sentiwiki.copernicus.eu/web/s2-mission).

- German Aerospace Center (DLR): Provides WSF3DV3 data layer that is used for the building height calculation process (https://geoservice.dlr.de/web/maps/eoc:wsf3d) & Esch, Brzoska, Dech, Leutner, Palacios-Lopez, Metz-Marconcini, Marconcini, Roth and Zeidler, 2022: "World Settlement Footprint 3D - A first three-dimensional survey of the global building stock".

- Google-Microsoft Open Buildings (combined and published by VIDA): Publicly available data contain a catalogue of buildings with specific coordinates and polygons (i.e. shapes of the buildings) for any given country or region (https://source.coop/repositories/vida/google-microsoft-open-buildings/description).

- GHS Settlement Model Grid: Publicly available data downloaded as GeoTIF to categorize buildings into Urban or Rural categories (https://human-settlement.emergency.copernicus.eu/download.php?ds=smod) || Schiavina, Marcello; Melchiorri, Michele; Pesaresi, Martino (2023): GHS-SMOD R2023A - GHS settlement layers, application of the Degree of Urbanisation methodology (stage I) to GHS-POP R2023A and GHS-BUILT-S R2023A, multitemporal (1975-2030). European Commission, Joint Research Centre (JRC) [Dataset] doi: 10.2905/A0DF7A6F-49DE-46EA-9BDE-563437A6E2BA PID: http://data.europa.eu/89h/a0df7a6f-49de-46ea-9bde-563437a6e2ba || Concept & Methodology: European Commission, and Statistical Office of the European Union, 2021. Applying the Degree of Urbanisation — A methodological manual to define cities, towns and rural areas for international comparisons — 2021 edition Publications Office of the European Union, 2021, ISBN 978-92-76-20306-3 doi: 10.2785/706535/

- Open Street Map (OSM): Publicly available data contain a catalogue of buildings with specific coordinates and polygons (i.e. shapes of the buildings). Data are downloaded as shapefiles (.shp) from geofabrik.de.

- Ookla’s Open Data: Open data sets available on a complimentary basis to help people make informed decisions around internet connectivity and internet speed (https://www.ookla.com/ookla-for-good/open-data).

- Overture Maps: Publicly available data contain a catalogue of buildings with specific coordinates and polygons (i.e. shapes of the buildings) (https://overturemaps.org/).

- Open Energy Maps: Providing building-level electricity access and consumption estimates for Kenya (https://www.openenergymaps.org/) & Lee, S.J., 2023: "Multimodal Data Fusion for Estimating Electricity Access and Demand" (Doctoral dissertation, Massachusetts Institute of Technology).


Note: The data is shared under the Open Data Commons Open Database License (ODbL) v1.0 license (https://opendatacommons.org/licenses/odbl/1-0/). 


This work was supported and built by SEforALL, DLR, Open Energy Maps, and Mahila Housing Trust through IBM Sustainability Accelerator program (2024). More info about the programme is available here https://www.ibm.com/impact/initiatives/ibm-sustainability-accelerator.

'''

with open('README.txt', 'w') as f:
    f.write(readme_txt_text)

In [4]:
# Import necessary libraries
import geopandas as gpd

import pandas as pd
import numpy as np
import shapely
from collections import Counter
import jaydebeapi as jdbc
import jpype
import os
from tqdm import tqdm

from botocore.client import Config
from pyproj import Geod
import ibm_boto3
import zipfile

In [5]:
# init S3 client in order to work with last tiff file version
cos_client = ibm_boto3.client(service_name='s3',
                              ibm_api_key_id=config["COS_APIKEY"],
                            #   ibm_auth_endpoint=config["COS_AUTH_ENDPOINT_URL"],
                              config=Config(signature_version='oauth'),
                              endpoint_url=config["COS_ENDPOINT_URL"])
geod = Geod(ellps="WGS84")

In [ ]:
# connect to the IBM DB2 function
def connect_to_db():

    jar = 'db2jcc4.jar'
    os.environ['CLASSPATH'] = jar

    args='-Djava.class.path=%s' % jar
    jvm_path = jpype.getDefaultJVMPath()
    try:
        jpype.startJVM(jvm_path, args)
    except Exception as e:
        print('startJVM exception: ', e)
        
    if jpype.isJVMStarted() and not jpype.isThreadAttachedToJVM():
        jpype.attachThreadToJVM()
        jpype.java.lang.Thread.currentThread().setContextClassLoader(jpype.java.lang.ClassLoader.getSystemClassLoader())
        
    # create JDBC connection
    conn = jdbc.connect(
                'com.ibm.db2.jcc.DB2Driver',
                config['DB2_CONNECTION_STRING'],
                [config["DB2_USERNAME"], config["DB2_PASSWORD"]],
                'db2jcc4.jar')
    
    return conn

conn = connect_to_db()
cursor = conn.cursor()

In [ ]:
# filepath that defines the source of districts polygons
# in district_boundaries folder here are shapefiles and geojsons that defines administrative bouders for selected region
# Maharashtra districts are represented by geojson from source https://github.com/HindustanTimesLabs/shapefiles/blob/master/state_ut/maharashtra/district/maharashtra_district.json
# Kenya counties are repesented by shapefile
filepath = 'boundaries/ke_county.shp'

if '.shp'in filepath:
    shapefile = gpd.read_file(filepath)
    # rename  county column in dataframe
    shapefile = shapefile.rename(columns={'county': 'district'})

elif '.json'in filepath and 'Maharashtra' in filepath:
    shapefile = gpd.read_file(open(filepath))
    # add country column
    shapefile['country'] = ['Maharashtra' for _ in  range(len(shapefile))]

# tablenames mapping for corresponding region
get_tablename = {
    'Kenya': 'FEATURES_DB_VIDA_EXTENDED',
    'Maharashtra': 'FEATURES_DB_MAHARASHTRA'
}

# get SQL tablename from which in next steps data will be queried
sql_tablename = get_tablename[shapefile.country.iloc[0]]
sql_tablename

In [ ]:
shapefile = shapefile[shapefile.district.isin(['Lamu', 'Makueni'])]
shapefile

In [9]:
def fetch_builings_in_bbox(cursor, lon_min, lon_max, lat_min, lat_max):
    '''
        This particular function is aimed for obtating all entries from defined rectangle for selected SQL table
    '''

    # fetch column names and their datatypes from defined SQL table
    sql = f'''SELECT 
                COLUMN_NAME, 
                TYPE_NAME
                FROM "SYSIBM"."SQLCOLUMNS"
                WHERE TABLE_SCHEM = 'USER1' AND TABLE_NAME = '{sql_tablename}' '''

    cursor.execute(sql)
    types_data = cursor.fetchall()

    typesmapper = {
        'VARCHAR': str, 
        'DOUBLE': float, 
        'INTEGER': int, 
        'SMALLINT':int
        }

    typesmapper = {str(i[0]).lower() :typesmapper.get(i[1]) for i in types_data}
    
    columns = list(typesmapper.keys())

    # sql statement for selecting entries by defined rectangle boundaries
    sql = f"""
        SELECT {', '.join(columns)} FROM USER1.{sql_tablename}
        WHERE 
            (LATITUDE >= {lat_min}) AND 
            (LATITUDE <= {lat_max}) AND 
            (LONGITUDE >= {lon_min}) AND 
            (LONGITUDE <= {lon_max})
        """
    
    try:
        cursor.execute(sql)
        data = cursor.fetchall()
    except Exception as e:
        print(f"Fetch items error occured: {e}")
        print("Reconnevting to the database try again...")

        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute(sql)
        data = cursor.fetchall()
    finally:
    #     # reshape obtained data to the GeoDataFrame
        df = pd.DataFrame(data=data, columns=columns)
        
        for col in columns:
            df[col] = df[col].astype(typesmapper[col], errors='ignore')

        return df


def create_and_upload_geojson(df, county_metadata):
    
    # fill None values in certain columns, in order not to get error on frontend side
    df['ml_confidence'] = df['ml_confidence'].fillna('-')
    df['height'] = df['height'].fillna('-')
    df['floors'] = df['floors'].fillna('-')
    df['osm_type'] = df['osm_type'].fillna('')
    df['gfa_in_meters'] = df['gfa_in_meters'].fillna('-')
    df['building_faces'] = df['building_faces'].fillna('-')
    df['perimeter_in_meters'] = df['perimeter_in_meters'].fillna('-')
    df['elec_access_percent'] = df['elec_access_percent'].fillna('-')
    df['elec_consumption_kwh_month'] = df['elec_consumption_kwh_month'].fillna('-')
    df['elec_consumption_std_kwh_month'] = df['elec_consumption_std_kwh_month'].fillna('-')

    if type(county_metadata.geometry) == shapely.geometry.multipolygon.MultiPolygon:
        geometry = shapely.concave_hull(county_metadata.geometry, ratio=1)
        county_coordinates = geometry.exterior.coords._coords.tolist()
        county_area = abs(geod.geometry_area_perimeter(geometry)[0])
    else:
        county_coordinates = county_metadata.geometry.exterior.coords._coords.tolist()
        county_area = abs(geod.geometry_area_perimeter(county_metadata.geometry)[0])

    res_nonres_stats = dict(Counter(df['classification_type']))
    rural_urban_stats = dict(Counter(df['urban_split']))

    county_properties = {
        'count_of_buildings': len(df),
        'count_of_buildings_res': res_nonres_stats['res'],
        'count_of_buildings_nonRes': res_nonres_stats['non-res'],
        'square_area_of_county': county_area,
        'square_area_of_buildings': df.area_in_meters.sum(),
        'square_area_res': df[df['classification_type'] == 'res'].area_in_meters.sum(),
        'square_area_nonRes': df[df['classification_type'] == 'non-res'].area_in_meters.sum(),
        'model_confidence_res': df[(df['classification_type'] == 'res') & (df['classification_source'] == 'classification_model')].ml_confidence.mean(),
        'model_confidence_nonRes': 1 - df[(df['classification_type'] == 'non-res') & (df['classification_source'] == 'classification_model')].ml_confidence.mean(),
        'height_avg': df.height.mean(),
        'height_avg_res': df[df['classification_type'] == 'res'].height.mean(),
        'height_avg_nonRes': df[df['classification_type'] == 'non-res'].height.mean(),
        'county_polygon_coordinates': county_coordinates
    }

        
    if 'Rural' in rural_urban_stats.keys():
        county_properties['rural'] = rural_urban_stats['Rural']

    if 'Urban' in rural_urban_stats.keys():
        county_properties['urban'] = rural_urban_stats['Urban']

    if 'Suburban' in rural_urban_stats.keys():
        county_properties['suburban'] = rural_urban_stats['Suburban']
    

    features = []
    for row in df.itertuples():
        try:
            polygon = shapely.from_wkt(row.polygon_coordinates)
            feature = {
                "type": "Feature",
                "properties": {
                    "latitude": row.latitude,
                    "longitude": row.longitude,
                    "height": row.height,
                    "area_in_meters": row.area_in_meters,
                    "classification_type": row.classification_type,
                    "type_source": row.classification_source,
                    "footprint_source": row.footprint_source,
                    "urban_split": row.urban_split,
                    "ghsl_smod": row.ghsl_smod,
                    "floors": row.floors,
                    "osm_type": row.osm_type,
                    "gfa_in_meters": row.gfa_in_meters,
                    "building_faces": row.building_faces,
                    "perimeter_in_meters": row.perimeter_in_meters,
                    "elec_access_percent": row.elec_access_percent,
                    "elec_consumption_kwh_month": row.elec_consumption_kwh_month,
                    "elec_consumption_std_kwh_month": row.elec_consumption_std_kwh_month
                    },  
                "geometry": {
                    "coordinates": [polygon.exterior.coords._coords.tolist()],
                    "type": "Polygon"
                }
            }

            if row.classification_source == 'classification_model':
                if row.classification_type == 'res':
                    feature['properties']['ml_confidence'] = round(row.ml_confidence, 5)
                else:
                    feature['properties']['ml_confidence'] = round(1 - row.ml_confidence, 5)

            if row.footprint_source == 'osm':
                feature['properties']['osm_id'] = row.osm_id


            features.append(feature)

        except Exception as e:
            print(e)

    try:
        geojson = {
            "type": "FeatureCollection",
            "county_properties": county_properties,
            "features": features
        }

        filename = f'{county_metadata.country}_{county_metadata.district}.geojson'
        file_path = f'{filename}'
        with open(file_path, "w") as outfile: 
            json.dump(geojson, outfile)

        cos_client.upload_file(Filename=file_path,Bucket=config["PRECREATED_GEOJSON_BUCKET"],Key=filename)
        print(f'Json {filename} successfully uploaded to the COS {config["PRECREATED_GEOJSON_BUCKET"]} bucket')
        
        zip_filename = f'{county_metadata.country}_{county_metadata.district}.zip'
        zf = zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED)
        zf.write(file_path)
        zf.write("README.txt")
        zf.close()
        cos_client.upload_file(Filename=zip_filename,Bucket=config["PRECREATED_GEOJSON_BUCKET"],Key=zip_filename)
        print(f'Zip {zip_filename} successfully uploaded to the COS {config["PRECREATED_GEOJSON_BUCKET"]} bucket')
        
        os.remove(zip_filename)
        os.remove(filename)
        
        # return geojson
        
    except Exception as e:
        print(f'functoin create_and_upload_geojson error occurred: {e}')

In [10]:
last_processed_idx = -1

In [ ]:
# try to make geojsons folder
try:
    os.makedirs('geojsons')
except Exception as e:
    print(e)
    
    
# try to make geoparquets folder
try:
    os.makedirs('geoparquets')
except Exception as e:
    print(e)

In [12]:
def save_and_upload_geoparquet(df, county_metadata):
    
    try:
        # save to parquet
        filename = f'{county_metadata.country}_{county_metadata.district}.geoparquet'.replace("'", '').replace(" ", '_')
        file_path = f'geoparquets/{filename}'
        df.to_parquet(file_path)
        
        # upload to the IBM COS
        cos_client.upload_file(
            Filename=file_path,
            Bucket=config["SHARE_PARQUET_BUCKET"],
            Key=filename,
            ExtraArgs={
                    'ContentDisposition': 'attachment',
                    }
        )
        
        os.remove(file_path)
        
        print(f'{filename} succesfully uploaded to COS bucket: {config["SHARE_PARQUET_BUCKET"]}')
    except Exception as e:
        print(f'functoin save_and_upload_geoparquet error occurred: {e}')

In [ ]:
for idx, county_metadata in enumerate(shapefile.itertuples()):
    if idx > last_processed_idx:
        print()
        print(f'Processing county: {county_metadata.district} {idx+1} of 47')
        # (minx, miny, maxx, maxy)
        min_lon, min_lat, max_lon, max_lat = county_metadata.geometry.bounds

        df = fetch_builings_in_bbox(cursor, min_lon, max_lon, min_lat, max_lat)
        
        df['buildings_in_polygon'] = [county_metadata.geometry.contains(shapely.Point(row.longitude, row.latitude)) for row in tqdm(df.itertuples(), total=len(df), desc='Filtering buildings')]

        df = df[df.buildings_in_polygon == True]
        df = df.drop(['buildings_in_polygon'], axis=1)

        
        print(f'buildings in polygoon {len(df)}')
        save_and_upload_geoparquet(df, county_metadata)
        
        create_and_upload_geojson(df, county_metadata)
        
        
        last_processed_idx = idx
        

In [ ]:
subdistricts_mapping = {}

for idx, county_metadata in enumerate(shapefile.itertuples()):

    # print(idx)
    district = county_metadata.county.replace("'", '').replace(" ", '_')

    if county_metadata.country not in subdistricts_mapping.keys():
        subdistricts_mapping[county_metadata.country] = {}
    # subdistrict = county_metadata.sdtname.replace('(', '').replace(')', '').replace(' ', '_')
    # subdistrict_sufix = '' if county_metadata.Index[1] == 0 else f'_{county_metadata.Index[1] + 1}'

    filename = f'{county_metadata.country}_{district}.json'
    subdistricts_mapping[county_metadata.country][district] = filename
    # print({f'{county_metadata.country}': {f'{district}' : filename}})
        
subdistricts_mapping

file_path = f'geojson_subdistricts_map.json'
with open(file_path, "w") as outfile: 
    json.dump(subdistricts_mapping, outfile)

subdistricts_mapping